# Imports

Before starting the import into Spotify you need to go to Spotify Developers and make a new App. From there onwards you need to follow this Notebook.

In [32]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
# Make a file in your directory containing username, client_id and client_secret
import keys

In [33]:
song_ids = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/df_audio_features_5000_cleaned_whitespaces.csv')

In [34]:
song_ids.sample()

,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,id,html
2788,Mad World - Bomb A Drop - Mix Cut ...,Hardwell,0.642,0.961,5,-2.151,1,0.0509,0.00176,0.00709,0.0736,0.151,127.976,,153499,4,36jz4zix5d5mE3W3so8cLU,https://open.spotify.com/track/36jz4zix5d5mE3W...


In [35]:
song_ids.drop(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'duration_ms', 'time_signature'], inplace=True)

In [36]:
songss = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists.csv')

In [37]:
songs = songss.merge(song_ids, on=['name', 'artist'], how='left')

In [38]:
# Group by 'cluster' column and count observations
cluster_counts = songs.groupby('cluster').size().sort_values(ascending=False)

cluster_counts

cluster
16    209
22    173
26    171
37    168
10    168
12    163
23    151
36    151
5     150
0     138
11    137
6     137
19    135
14    133
8     131
4     128
13    127
30    123
35    120
3     120
20    117
7     115
21    108
2     107
27    105
9     103
31    100
18     99
15     99
41     92
38     91
24     88
28     86
1      86
46     78
43     76
49     71
32     69
42     68
39     67
17     67
40     65
34     56
47     48
25     45
33     43
29     42
45     39
44     38
48     34
dtype: int64

In [27]:
scope='playlist-modify-public'
username= keys.username
redirectUri = 'http://localhost:8080/callback'
client_id=keys.client_id
client_secret=keys.client_secret
playlist_name = 'My Once Playlist'
playlist_description = 'App for my bootcamp project Moosaic. Create playlists through clustering.'

In [28]:
token = SpotifyOAuth(scope=scope,username=username,client_id=client_id,
                     client_secret=client_secret, redirect_uri=redirectUri)

In [29]:
spotifyObject = spotipy.Spotify(auth_manager=token)

In [30]:
for clust in songs['cluster'].unique():
    playlist_name = f'Playlist Cluster {clust}'
    playlist_description = 'App for my bootcamp project Moosaic. Create playlists through clustering.'
    
    try:
        playlist_id = spotifyObject.user_playlist_create(user=username, name=playlist_name, 
                                                         description=playlist_description, public=True)
        print(f"Playlist created with ID: {playlist_id}")
    except Exception as e:
        print(f"An error occurred: {e}")

    #spotifyObject.user_playlist_create(user=username, name=playlist_name, description=playlist_description, public=True)
    
    list_of_songs = []

    # Filter songs for cluster 0
    cluster_cluster_songs = songs[songs['cluster'] == cluster]

    for i, row in cluster_cluster_songs.iterrows():
        list_of_songs.append(f'spotify:track:{row["id"]}')

    spotifyObject.user_playlist_add_tracks(user=username,playlist_id=playlist_id['id'],tracks=list_of_songs)

In [10]:
list_of_songs=[]
for i, row in songs.iterrows():
    list_of_songs.append(f'spotify:track:{row["id"]}')

In [ ]:
list_of_songs

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# The maximum number of tracks you can add per request
MAX_TRACKS_PER_REQUEST = 100

# Split the list_of_songs into chunks
song_chunks = list(chunks(list_of_songs, MAX_TRACKS_PER_REQUEST))

# Add each chunk to the playlist
for chunk in song_chunks:
    spotifyObject.user_playlist_add_tracks(user=username, playlist_id=playlist_id['id'], tracks=chunk)

In [ ]:
spotifyObject.user_playlist_add_tracks(user=username,playlist_id=playlist_id['id'],tracks=list_of_songs)

# Try otherwise

In [ ]:
# import spotipy
# import pandas as pd
# from spotipy.oauth2 import SpotifyOAuth
# import keys

# songs = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists.csv')

# # Spotify API credentials (You'll get these from the Spotify Developer Dashboard)
# SPOTIPY_CLIENT_ID = keys.client_id
# SPOTIPY_CLIENT_SECRET = keys.client_secret
# SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

# # Setting up spotipy with user authentication
# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
#     client_id=SPOTIPY_CLIENT_ID,
#     client_secret=SPOTIPY_CLIENT_SECRET,
#     redirect_uri=SPOTIPY_REDIRECT_URI,
#     scope="playlist-modify-public"  # This scope allows creating public playlists
# ))

# # Your user ID (You can also retrieve this programmatically after authentication)
# username = keys.username

# # Details for the new playlist
# playlist_name = "My New Playlist"
# playlist_description = "This is a playlist created using the Spotify Web API."

# try:
#     # Create a new playlist
#     playlist = sp.user_playlist_create(user=username, name=playlist_name, 
#                                        description=playlist_description, public=True)
#     print(f"Playlist created with ID: {playlist['id']}")
# except Exception as e:
#     print(f"An error occurred: {e}")
